<a href="https://colab.research.google.com/github/zhenkanglau/Wavenet-implement-and-stock-prediction/blob/master/wavenet_implement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import TensorDataset,Dataset,IterableDataset


data.zip  sample_data


In [ ]:
!wget https://download.pytorch.org/tutorial/data.zip

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self,input_size,hidden_size):
      super(EncoderRNN,self).__init__()
      self.input_size=input_size
      self.hidden_size=hidden_size
      self.embedding=nn.Embedding(input_size,output_size)
      self.gru=nn.GRU(hidden_size,hidden_size)
    
    def forward(self,input,hidden):
      embedded=self.embedding(input).view(1,1,-1)
      output = embedded
      output, hidden = self.gru(output,hidden)
      return output,hidden
    
    def initHidden(self):
      return torch.zeros(1,1,self.hidden_size)

In [ ]:
class DecoderRNN(nn.Module):
  def __init__(self,output_size,hidden_size):
    super(DecoderRNN,self).__init__()
    self.hidden_size=hidden_size
    self.output_size=output_size

    self.embedding=nn.Embedding(input_size,output_size)
    self.gru=nn.Gru(hidden_size,output_size)
    self.out=nn.Linear(hidden_size,output_size)
    self.softmax=nn.LogSoftmax(dim=1)
  
  def forward(self,input,hidden):
    output = self.embedding(input).view(1,1,-1)
    output = nn.relu(output)
    output,hidden = self.gru(output,hidden)
    output = self.softmax(self.out(output[0]))
    return output,hidden
  
  def initHidden(self):
    return torch.zeros(1,1,self.hidden_size)


In [ ]:
batch_size=3
learning_rate=0.001
epoch=1000




In [ ]:
a=torch.tensor([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16],dtype=torch.float32)

In [ ]:
a

tensor([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14.,
        15., 16.], dtype=torch.float64)

In [ ]:
[2**l for l in range(1, 10+1)]

[2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]

In [ ]:
class CausalConv1d(torch.nn.Conv1d):
    def __init__(self,
                 in_channels,
                 out_channels,
                 kernel_size,
                 stride=1,
                 dilation=1,
                 groups=1,
                 bias=True):

        super(CausalConv1d, self).__init__(
            in_channels,
            out_channels,
            kernel_size=kernel_size,
            stride=stride,
            padding=0,
            dilation=dilation,
            groups=groups,
            bias=bias)
        
        self.__padding = (kernel_size - 1) * dilation
        
    def forward(self, input):
        return super(CausalConv1d, self).forward(F.pad(input, (self.__padding, 0)))#(padding_left,padding_right)

In [ ]:
class CausalConv1d(torch.nn.Conv1d):
    def __init__(self,
                 in_channels,
                 out_channels,
                 kernel_size,
                 stride=1,
                 dilation=1,
                 groups=1,
                 bias=True):
        self.__padding = (kernel_size - 1) * dilation

        super(CausalConv1d, self).__init__(
            in_channels,
            out_channels,
            kernel_size=kernel_size,
            stride=stride,
            padding=self.__padding,
            dilation=dilation,
            groups=groups,
            bias=bias)

    def forward(self, input):
        result = super(CausalConv1d, self).forward(input)
        if self.__padding != 0:
            return result[:, :, :-self.__padding]
        return result

In [ ]:
class ResidualBlock(nn.Module):
  def __init__(self):
    super(ResidualBlock,self).__init__()
  
  def forward(self,x):
    g=torch.sigmoid(x)
    h=torch.tanh(x)
    f=g*h
    res=f+x

    return res,f

In [ ]:
 res=ResidualBlock()
 b,c=res(a)
 print(b)
 print(c)

tensor([ 1.5568,  2.8491,  3.9479,  4.9814,  5.9932,  6.9975,  7.9991,  8.9997,
         9.9999, 11.0000, 12.0000, 13.0000, 14.0000, 15.0000, 16.0000, 17.0000])
tensor([0.5568, 0.8491, 0.9479, 0.9814, 0.9932, 0.9975, 0.9991, 0.9997, 0.9999,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


In [ ]:
class WaveNet(nn.Module):
    def __init__(self):
        super(WaveNet, self).__init__()
        #self.dconv1=CausalConv1d(1,1,3,1,1,bias=False)
        self.dconv1 = nn.Conv1d(1, 1, kernel_size=3, padding=int((1*(3-1))/2), dilation=1,bias=False)
        #self.dconv2=CausalConv1d(1,1,3,1,dilation=2,bias=False)
        self.dconv2 = nn.Conv1d(1, 1, kernel_size=3, padding=int((2*(3-1))/2), dilation=2,bias=False)
        self.dconv3=CausalConv1d(1,1,3,1,dilation=4,bias=False)
        self.res1=ResidualBlock()
        self.res2=ResidualBlock()
    def forward(self,x):
      y = self.dconv1(x)
      #y,skip = self.res1(y)
      #y = self.dconv2(y)
     # y,skip = self.res2(y)
      #totalskip=totalskip+skip
      #y = self.dconv3(y)
      #y,skip = ResidualBlock(y)
      #totalskip=totalskip+skip
      return y
    


In [ ]:
kernel_size=3
dilation_rate=2
(dilation_rate*(kernel_size-1))/2


2.0

In [ ]:
(4 - 1) // 2

1

In [ ]:
F.pad(a,(2,0))#(padding_left,padding_right)

tensor([ 0.,  0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
        13., 14., 15., 16.])

In [ ]:
a.view(1,1,-1)

tensor([[[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16]]])

In [ ]:
wavenet.dconv1.weight

tensor([[[1., 2., 3.]]], grad_fn=<MulBackward0>)

In [ ]:
wavenet=WaveNet()
torch.nn.init.ones_(wavenet.dconv1.weight)
torch.nn.init.ones_(wavenet.dconv2.weight)
torch.nn.init.ones_(wavenet.dconv3.weight)
wavenet(a.view(1,1,-1))

tensor([[[10., 14., 18., 22., 26., 30., 34., 38., 42., 46., 50., 54., 58.]]],
       grad_fn=<SqueezeBackward1>)